# Zero-Shot Cross-Lingual Natural Language Inference with Bi-Encoders and Cross-Encoders Training Pipeline

In [ ]:
# 🧱 Step 1: Install Required Libraries
# Ensure all required libraries are installed
import sys
import subprocess

required_libraries = ["transformers", "datasets", "torch", "scikit-learn"]
for lib in required_libraries:
    try:
        __import__(lib)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib])

In [ ]:
# 🧠 Step 2: Import Required Libraries

# Standard Library Imports
from collections import Counter
import time

# Third-Party Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.utils import resample
from datasets import load_dataset
import evaluate
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    get_linear_schedule_with_warmup,
)
from tqdm.auto import tqdm

In [ ]:
# 📥 Step 3: Load Datasets (MNLI and XNLI)
def load_datasets():
    """
    Load MNLI and XNLI datasets.
    Returns:
        mnli_dataset: MNLI dataset
        xnli_es_dataset: XNLI Spanish dataset
        xnli_de_dataset: XNLI German dataset
    """
    try:
        mnli_dataset = load_dataset("glue", "mnli")
        xnli_es_dataset = load_dataset("xnli", "es")
        xnli_de_dataset = load_dataset("xnli", "de")
    except Exception as e:
        print(f"Error loading datasets: {e}")
        raise
    return mnli_dataset, xnli_es_dataset, xnli_de_dataset

mnli_dataset, xnli_es_dataset, xnli_de_dataset = load_datasets()
print("MNLI Dataset:", mnli_dataset)
print("XNLI Spanish Dataset:", xnli_es_dataset)
print("XNLI German Dataset:", xnli_de_dataset)

In [ ]:
# 📊 Step 4: Data Inspection and Class Distribution Analysis
def analyze_class_distribution(dataset, split_name):
    """
    Analyze and visualize class distribution in a dataset split.
    Args:
        dataset: The dataset to analyze.
        split_name: The split name (e.g., "train", "test").
    """
    labels = dataset[split_name]["label"]
    label_counts = Counter(labels)
    total_samples = len(labels)
    label_percentages = {label: count / total_samples * 100 for label, count in label_counts.items()}
    df = pd.DataFrame({
        "Label": ["Entailment", "Neutral", "Contradiction"],
        "Count": [label_counts.get(0, 0), label_counts.get(1, 0), label_counts.get(2, 0)],
        "Percentage": [label_percentages.get(0, 0), label_percentages.get(1, 0), label_percentages.get(2, 0)]
    })
    print(f"\n--- Class Distribution of {split_name} ---")
    print(df.to_markdown(index=False))

    plt.figure(figsize=(8, 6))
    sns.barplot(x=df["Label"], y=df["Percentage"])
    plt.title(f"Class Distribution of {split_name}")
    plt.ylabel("Percentage")
    plt.show()

# Analyze class distribution for each dataset split
analyze_class_distribution(mnli_dataset, "train")
analyze_class_distribution(mnli_dataset, "validation_matched")
analyze_class_distribution(xnli_es_dataset, "test")
analyze_class_distribution(xnli_de_dataset, "test")

In [ ]:
# 📏 Step 5: Tokenization Check and Setting `max_length`
def analyze_token_lengths(dataset, tokenizer, split_name):
    """
    Analyze token lengths in a dataset split.
    Args:
        dataset: The dataset to analyze.
        tokenizer: The tokenizer to use.
        split_name: The split name (e.g., "train").
    Returns:
        max_length: Recommended maximum token length.
    """
    def get_token_lengths(examples):
        return {
            "premise_length": [len(tokenizer.encode(p)) for p in examples["premise"]],
            "hypothesis_length": [len(tokenizer.encode(h)) for h in examples["hypothesis"]],
        }

    # Apply the function and remove only the original columns
    token_lengths = dataset[split_name].map(
        get_token_lengths,
        batched=True,
        remove_columns=["premise", "hypothesis"]
    )

    # Extract the lengths from the dataset
    premise_lengths = token_lengths["premise_length"]
    hypothesis_lengths = token_lengths["hypothesis_length"]

    all_lengths = np.array(premise_lengths + hypothesis_lengths)

    print(f"\n--- Token Length Statistics ({split_name}) ---")
    df_lengths = pd.DataFrame({
        "Metric": ["Mean", "Median", "P90", "P95", "Max"],
        "Premise Length": [np.mean(premise_lengths), np.median(premise_lengths), np.percentile(premise_lengths, 90), np.percentile(premise_lengths, 95), np.max(premise_lengths)],
        "Hypothesis Length": [np.mean(hypothesis_lengths), np.median(hypothesis_lengths), np.percentile(hypothesis_lengths, 90), np.percentile(hypothesis_lengths, 95), np.max(hypothesis_lengths)],
        "Combined Length": [np.mean(all_lengths), np.median(all_lengths), np.percentile(all_lengths, 90), np.percentile(all_lengths, 95), np.max(all_lengths)],
    })
    print(df_lengths.to_markdown(index=False))

    sns.histplot(all_lengths, bins=50)
    plt.title(f"Distribution of Combined Token Lengths ({split_name})")
    plt.xlabel("Token Length")
    plt.ylabel("Frequency")
    plt.axvline(128, color='r', linestyle='--', label='max_length = 128')
    plt.legend()
    plt.show()

    max_length = 128  # Based on analysis
    return max_length

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer Loaded:", tokenizer)

max_length = analyze_token_lengths(mnli_dataset, tokenizer, "train")
print(f"Recommended max_length: {max_length}")

In [ ]:
# 🧹 Step 6: Tokenizer and Preprocessing Function
def tokenize_batch(batch):
    """
    Tokenizes premise and hypothesis pairs in a batch.
    Args:
        batch (dict): A batch containing 'premise' and 'hypothesis' fields.
    Returns:
        dict: Tokenized inputs with padding and truncation applied.
    """
    return tokenizer(batch["premise"], batch["hypothesis"],
                     padding="max_length", truncation=True, max_length=max_length)

def rename_label(example):
    """
    Renames the 'label' field to 'labels' for compatibility with PyTorch.
    Args:
        example (dict): A single example containing a 'label' field.
    Returns:
        dict: Example with the 'label' field renamed to 'labels'.
    """
    return {"labels": example["label"]}

In [ ]:
# 🏷️ Step 7: Apply Tokenization to Datasets
def preprocess_and_format(dataset, split_name, columns_to_remove):
    """
    Tokenizes, renames labels, and formats a dataset split for PyTorch.
    Args:
        dataset (Dataset): The dataset to preprocess.
        split_name (str): The split name (e.g., "train", "test").
        columns_to_remove (list): List of columns to remove after tokenization.
    Returns:
        Dataset: Preprocessed and formatted dataset.
    """
    encoded_dataset = dataset[split_name].map(tokenize_batch, batched=True)
    encoded_dataset = encoded_dataset.map(rename_label)
    encoded_dataset = encoded_dataset.remove_columns(columns_to_remove)
    encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids", "labels"])
    return encoded_dataset

mnli_encoded_train = preprocess_and_format(mnli_dataset, "train", ["premise", "hypothesis", "label", "idx"])
print("MNLI Training set tokenized and formatted:", mnli_encoded_train)

mnli_encoded_validation = preprocess_and_format(mnli_dataset, "validation_matched", ["premise", "hypothesis", "label", "idx"])
print("MNLI Validation set tokenized and formatted:", mnli_encoded_validation)

xnli_es_encoded = preprocess_and_format(xnli_es_dataset, "test", ["premise", "hypothesis", "label"])
print("XNLI Spanish test set tokenized and formatted:", xnli_es_encoded)

xnli_de_encoded = preprocess_and_format(xnli_de_dataset, "test", ["premise", "hypothesis", "label"])
print("XNLI German test set tokenized and formatted:", xnli_de_encoded)

In [ ]:
# ✂️ Step 8: Create Baseline Training Subset
def create_training_subset(dataset, test_size=0.9, seed=42):
    """
    Creates a smaller training subset for baseline experiments.
    Args:
        dataset (Dataset): The full training dataset.
        test_size (float): Proportion of the dataset to use as the test set.
        seed (int): Random seed for reproducibility.
    Returns:
        Dataset: Training subset.
    """
    return dataset.train_test_split(test_size=test_size, seed=seed)["train"]

train_subset = create_training_subset(mnli_encoded_train)
print("Baseline training subset created:", train_subset)

In [ ]:
# 📦 Step 9: DataLoader Creation Utility
def create_dataloader(encoded_dataset, batch_size=16, shuffle=False):
    """
    Creates a PyTorch DataLoader for a given dataset.
    Args:
        encoded_dataset (Dataset): The dataset to load.
        batch_size (int): Number of samples per batch.
        shuffle (bool): Whether to shuffle the dataset.
    Returns:
        DataLoader: PyTorch DataLoader.
    """
    return DataLoader(encoded_dataset, batch_size=batch_size, shuffle=shuffle)

In [ ]:
# 🧪 Step 10: DataLoader Creation for Baseline
baseline_train_dataloader = create_dataloader(train_subset, batch_size=64, shuffle=True)
baseline_val_dataloader = create_dataloader(mnli_encoded_validation, batch_size=64) # Use the full validation set for evaluation
print("Baseline training DataLoader created:", baseline_train_dataloader)
print("Baseline validation DataLoader created:", baseline_val_dataloader)


In [ ]:
# ⚡️ Step 11: Confirm GPU Availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\nUsing device:", device)


In [ ]:
# 🎯 Step 12: Define Pretrained Multilingual Cross-Encoder Model
# Load a pretrained multilingual BERT model with a classification head for 3 labels (Entailment, Neutral, Contradiction)
try:
    model = AutoModelForSequenceClassification.from_pretrained(
        "bert-base-multilingual-cased",
        num_labels=3,
    ).to(device)
    print("Pretrained multilingual BERT model loaded with classification head and moved to device.")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

In [ ]:
# 📈 Step 13: Define Evaluation Metrics
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """
    Computes evaluation metrics for model predictions.
    Args:
        eval_pred (tuple): A tuple containing logits and true labels.
    Returns:
        dict: A dictionary with accuracy, precision, recall, and F1 score.
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
print("Evaluation metrics defined.")

In [ ]:
# ⚙️ Step 14: Define Training Arguments for Baseline
# Define key training parameters
learning_rate = 2e-5
num_train_epochs = 1
batch_size = 64

baseline_training_args = TrainingArguments(
    output_dir="./results-baseline-crossencoder",  # Directory to save model checkpoints
    evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    save_total_limit=1,  # Keep only the most recent checkpoint
    learning_rate=learning_rate,  # Learning rate for the optimizer
    per_device_train_batch_size=batch_size,  # Batch size for training
    per_device_eval_batch_size=batch_size,  # Batch size for evaluation
    num_train_epochs=num_train_epochs,  # Number of training epochs
    weight_decay=0.01,  # Weight decay for regularization
    logging_dir="./logs-baseline",  # Directory for logging
    logging_strategy="epoch",  # Log metrics at the end of each epoch
    report_to="none"  # Disable reporting to external services
)
print("Baseline training arguments defined:", baseline_training_args)

In [ ]:
# 🚀 Step 15: Initialize and Train Baseline Cross-Encoder (Corrected)
try:
    baseline_trainer = Trainer(
        model=model,
        args=baseline_training_args,
        train_dataset=train_subset,
        eval_dataset=mnli_encoded_validation,  # Use the encoded Dataset here
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    print("Baseline Trainer initialized:", baseline_trainer)

    baseline_results = baseline_trainer.train()
    print("Baseline training completed:", baseline_results)
except Exception as e:
    print(f"Error during training: {e}")
    raise

In [ ]:
# 🧪 Step 16: Evaluate Baseline on MNLI Validation and Print Metrics
try:
    baseline_eval_results = baseline_trainer.evaluate()

    print("\n--- Baseline Evaluation Results (MNLI Validation) ---")
    df_baseline_metrics = pd.DataFrame([baseline_eval_results])
    print(df_baseline_metrics[["eval_accuracy", "eval_precision", "eval_recall", "eval_f1"]].to_markdown(index=False))

    print("\n--- Baseline Evaluation Dictionary ---")
    print(baseline_eval_results)
except Exception as e:
    print(f"Error during evaluation: {e}")
    raise

In [ ]:
# ⚙️ Step 17: Define Training Arguments for Full Model
# Define key training parameters
learning_rate = 2e-5
num_train_epochs = 3
batch_size = 64

full_model_training_args = TrainingArguments(
    output_dir="./results-full-crossencoder",  # Directory to save model checkpoints
    evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    save_total_limit=1,  # Keep only the most recent checkpoint
    learning_rate=learning_rate,  # Learning rate for the optimizer
    per_device_train_batch_size=batch_size,  # Batch size for training
    per_device_eval_batch_size=batch_size,  # Batch size for evaluation
    num_train_epochs=num_train_epochs,  # Number of training epochs
    weight_decay=0.01,  # Weight decay for regularization
    logging_dir="./logs-full",  # Directory for logging
    logging_strategy="epoch",  # Log metrics at the end of each epoch
    report_to="none"  # Disable reporting to external services
)
print("Full model training arguments defined:", full_model_training_args)

In [ ]:
# 🚀 Step 18: Initialize and Train Full Cross-Encoder
try:
    full_model_trainer = Trainer(
        model=model,  # We are using the same model instance, which will continue training
        args=full_model_training_args,
        train_dataset=mnli_encoded_train,  # Use the full encoded training set
        eval_dataset=mnli_encoded_validation,  # Use the full encoded validation set
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    print("Full model Trainer initialized:", full_model_trainer)

    full_model_results = full_model_trainer.train()
    print("Full model training completed:", full_model_results)

    # 🧪 Evaluate Full Model on MNLI Validation Set
    full_model_eval_results_mnli = full_model_trainer.evaluate()

    print("\n--- Full Model Evaluation Results (MNLI Validation) ---")
    df_full_model_metrics_mnli = pd.DataFrame([full_model_eval_results_mnli])
    print(df_full_model_metrics_mnli[["eval_accuracy", "eval_precision", "eval_recall", "eval_f1"]].to_markdown(index=False))

    print("\n--- Full Model Evaluation Dictionary (MNLI Validation) ---")
    print(full_model_eval_results_mnli)
except Exception as e:
    print(f"Error during training or evaluation: {e}")
    raise

In [ ]:
# 🌍 Step 19: Evaluate Full Model on XNLI Spanish Test Set
try:
    full_model_eval_results_xnli_es = full_model_trainer.evaluate(xnli_es_encoded, metric_key_prefix="xnli_es")

    print("\n--- Full Model Evaluation Results (XNLI Spanish Test) ---")
    df_full_model_metrics_xnli_es = pd.DataFrame([full_model_eval_results_xnli_es])
    print(df_full_model_metrics_xnli_es[["xnli_es_accuracy", "xnli_es_precision", "xnli_es_recall", "xnli_es_f1"]].to_markdown(index=False))

    print("\n--- Full Model Evaluation Dictionary (XNLI Spanish Test) ---")
    print(full_model_eval_results_xnli_es)

    # Save Cross-Encoder Metrics for XNLI Spanish
    cross_encoder_xnli_es_metrics = {
        "accuracy": full_model_eval_results_xnli_es["xnli_es_accuracy"],
        "precision": full_model_eval_results_xnli_es["xnli_es_precision"],
        "recall": full_model_eval_results_xnli_es["xnli_es_recall"],
        "f1": full_model_eval_results_xnli_es["xnli_es_f1"],
    }
    print("\n--- Saved Cross-Encoder Metrics (XNLI Spanish) ---")
    print(cross_encoder_xnli_es_metrics)
except Exception as e:
    print(f"Error during evaluation on XNLI Spanish Test Set: {e}")
    raise

In [ ]:
# 🇩🇪 Step 20: Evaluate Full Model on XNLI German Test Set
try:
    full_model_eval_results_xnli_de = full_model_trainer.evaluate(xnli_de_encoded, metric_key_prefix="xnli_de")

    print("\n--- Full Model Evaluation Results (XNLI German Test) ---")
    df_full_model_metrics_xnli_de = pd.DataFrame([full_model_eval_results_xnli_de])
    print(df_full_model_metrics_xnli_de[["xnli_de_accuracy", "xnli_de_precision", "xnli_de_recall", "xnli_de_f1"]].to_markdown(index=False))

    print("\n--- Full Model Evaluation Dictionary (XNLI German Test) ---")
    print(full_model_eval_results_xnli_de)

    # Save Cross-Encoder Metrics for XNLI German
    cross_encoder_xnli_de_metrics = {
        "accuracy": full_model_eval_results_xnli_de["xnli_de_accuracy"],
        "precision": full_model_eval_results_xnli_de["xnli_de_precision"],
        "recall": full_model_eval_results_xnli_de["xnli_de_recall"],
        "f1": full_model_eval_results_xnli_de["xnli_de_f1"],
    }
    print("\n--- Saved Cross-Encoder Metrics (XNLI German) ---")
    print(cross_encoder_xnli_de_metrics)
except Exception as e:
    print(f"Error during evaluation on XNLI German Test Set: {e}")
    raise

In [ ]:
# 🛠️ Step 21: Initialize the Bi-Encoder Model
class BiEncoder(nn.Module):
    """
    Bi-Encoder model for generating embeddings for premise and hypothesis pairs.
    """
    def __init__(self, model_name):
        super(BiEncoder, self).__init__()
        try:
            self.transformer = AutoModel.from_pretrained(model_name)
            self.embedding_dim = self.transformer.config.hidden_size
        except Exception as e:
            print(f"Error initializing Bi-Encoder: {e}")
            raise

    def forward(self, input_ids, attention_mask, token_type_ids):
        """
        Forward pass for the Bi-Encoder.
        Args:
            input_ids: Tokenized input IDs.
            attention_mask: Attention mask for the input.
            token_type_ids: Token type IDs for the input.
        Returns:
            cls_embedding: Embedding of the [CLS] token.
        """
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        cls_embedding = outputs.pooler_output  # Extract the [CLS] token embedding
        return cls_embedding

bi_encoder_model_name = "bert-base-multilingual-cased"
bi_encoder = BiEncoder(bi_encoder_model_name)
print("Bi-Encoder Model Initialized:", bi_encoder)
print("Bi-Encoder Embedding Dimension:", bi_encoder.embedding_dim)

In [ ]:
# ⚙️ Step 22: Load and Encode MNLI Datasets for Bi-Encoder (Robust)

# Load the MNLI dataset
mnli_dataset = load_dataset("multi_nli")
mnli_train_dataset = mnli_dataset["train"]
mnli_val_dataset = mnli_dataset["validation_matched"]  # Using "matched" for validation

# Tokenization function for bi-encoder (separate premise and hypothesis)
def encode_bi_encoder_robust(examples):
    premises = examples["premise"]
    hypotheses = examples["hypothesis"]

    encoded_premises = tokenizer(premises, truncation=True, padding="max_length", max_length=100, return_tensors="pt")
    encoded_hypotheses = tokenizer(hypotheses, truncation=True, padding="max_length", max_length=100, return_tensors="pt")

    return {
        "premise_input_ids": encoded_premises["input_ids"],
        "premise_attention_mask": encoded_premises["attention_mask"],
        "premise_token_type_ids": encoded_premises["token_type_ids"],
        "hypothesis_input_ids": encoded_hypotheses["input_ids"],
        "hypothesis_attention_mask": encoded_hypotheses["attention_mask"],
        "hypothesis_token_type_ids": encoded_hypotheses["token_type_ids"],
        "labels": examples["label"]
    }

# Encode the MNLI training and validation datasets
encoded_mnli_train = mnli_train_dataset.map(encode_bi_encoder_robust, batched=True)
encoded_mnli_val = mnli_val_dataset.map(encode_bi_encoder_robust, batched=True)

# Set the format for PyTorch tensors and specify the columns to return
encoded_mnli_train.set_format("torch", columns=[
    "premise_input_ids", "premise_attention_mask", "premise_token_type_ids",
    "hypothesis_input_ids", "hypothesis_attention_mask", "hypothesis_token_type_ids", "labels"
])
encoded_mnli_val.set_format("torch", columns=[
    "premise_input_ids", "premise_attention_mask", "premise_token_type_ids",
    "hypothesis_input_ids", "hypothesis_attention_mask", "hypothesis_token_type_ids", "labels"
])

print("Encoded MNLI Training Dataset (Robust):", encoded_mnli_train)
print("Encoded MNLI Validation Dataset (Robust):", encoded_mnli_val)

In [ ]:
# ⚙️ Step 23: Create DataLoaders for Bi-Encoder Training
# Create DataLoaders for training and validation datasets
# Shuffle the training DataLoader to ensure randomness in batches
bi_encoder_train_dataloader = DataLoader(encoded_mnli_train, batch_size=32, shuffle=True)
bi_encoder_val_dataloader = DataLoader(encoded_mnli_val, batch_size=32)

print("Bi-Encoder Training DataLoader created with batch size 32.")
print("Bi-Encoder Validation DataLoader created with batch size 32.")

# Uncomment the following lines to inspect a batch
# for batch in bi_encoder_train_dataloader:
#     print("Batch keys:", batch.keys())
#     break

In [ ]:
# 🛠️ Step 24: Initialize the Bi-Encoder Classification Head
class BiEncoderClassificationHead(nn.Module):
    """
    Classification head for the Bi-Encoder model.
    Combines premise and hypothesis embeddings and predicts the label.
    """
    def __init__(self, embedding_dim, num_classes):
        super(BiEncoderClassificationHead, self).__init__()
        self.fc1 = nn.Linear(embedding_dim * 3, embedding_dim)  # Concatenate u, v, |u-v|
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(embedding_dim, num_classes)

    def forward(self, premise_embedding, hypothesis_embedding):
        """
        Forward pass for the classification head.
        Args:
            premise_embedding: Embedding of the premise.
            hypothesis_embedding: Embedding of the hypothesis.
        Returns:
            output: Logits for each class.
        """
        # Calculate the absolute difference between the embeddings
        abs_diff = torch.abs(premise_embedding - hypothesis_embedding)

        # Concatenate the premise embedding, hypothesis embedding, and their absolute difference
        concatenated_embedding = torch.cat((premise_embedding, hypothesis_embedding, abs_diff), dim=-1)

        # Pass through the fully connected layers
        output = self.fc1(concatenated_embedding)
        output = self.relu(output)
        output = self.fc2(output)
        return output

# Initialize the classification head
num_nli_labels = 3  # Entailment, Neutral, Contradiction
bi_encoder_classifier = BiEncoderClassificationHead(bi_encoder.embedding_dim, num_nli_labels)
print("Bi-Encoder Classification Head Initialized:", bi_encoder_classifier)

In [ ]:
# ⚙️ Step 25: Define Loss Function and Optimizer
# Cross-entropy loss is used for multi-class classification (Entailment, Neutral, Contradiction)
criterion = nn.CrossEntropyLoss()

# AdamW optimizer is used for training transformer-based models with weight decay
optimizer = optim.AdamW(
    list(bi_encoder.parameters()) + list(bi_encoder_classifier.parameters()), 
    lr=2e-5  # Learning rate
)

print("Loss Function:", criterion)
print("Optimizer:", optimizer)

In [ ]:
# ⚙️ Step 26: Define train_bi_encoder_step Function
def train_bi_encoder_step(model, classifier, optimizer, criterion, batch, device):
    """
    Performs a single training step for the Bi-Encoder model.

    Args:
        model (nn.Module): The Bi-Encoder model.
        classifier (nn.Module): The classification head for the Bi-Encoder.
        optimizer (torch.optim.Optimizer): Optimizer for updating model weights.
        criterion (nn.Module): Loss function for calculating the training loss.
        batch (dict): A batch of input data containing premise, hypothesis, and labels.
        device (torch.device): The device (CPU or GPU) to run the training step on.

    Returns:
        float: The loss value for the current training step.
    """
    # Set the model and classifier to training mode
    model.train()
    classifier.train()

    # Zero the gradients
    optimizer.zero_grad()

    # Move batch data to the specified device
    premise_input_ids = batch["premise_input_ids"].to(device)
    premise_attention_mask = batch["premise_attention_mask"].to(device)
    premise_token_type_ids = batch["premise_token_type_ids"].to(device)
    hypothesis_input_ids = batch["hypothesis_input_ids"].to(device)
    hypothesis_attention_mask = batch["hypothesis_attention_mask"].to(device)
    hypothesis_token_type_ids = batch["hypothesis_token_type_ids"].to(device)
    labels = batch["labels"].to(device)

    # Get embeddings from the Bi-Encoder for premise and hypothesis
    premise_embedding = model(premise_input_ids, premise_attention_mask, premise_token_type_ids)
    hypothesis_embedding = model(hypothesis_input_ids, hypothesis_attention_mask, hypothesis_token_type_ids)

    # Pass the embeddings through the classification head to get logits
    logits = classifier(premise_embedding, hypothesis_embedding)

    # Calculate the loss using the criterion
    loss = criterion(logits, labels)

    # Backpropagate the loss and update model weights
    loss.backward()
    optimizer.step()

    # Return the loss value for logging
    return loss.item()

print("Function 'train_bi_encoder_step' defined.")

In [ ]:
# ⚙️ Step 27: Define `evaluate_bi_encoder` Function
def evaluate_bi_encoder(model, classifier, dataloader, criterion, device):
    """
    Evaluates the Bi-Encoder model on a given dataset.

    Args:
        model (nn.Module): The Bi-Encoder model.
        classifier (nn.Module): The classification head for the Bi-Encoder.
        dataloader (DataLoader): DataLoader for the evaluation dataset.
        criterion (nn.Module): Loss function for calculating the evaluation loss.
        device (torch.device): The device (CPU or GPU) to run the evaluation on.

    Returns:
        tuple: A tuple containing:
            - avg_loss (float): Average loss over the dataset.
            - accuracy (float): Accuracy of the model.
            - precision (float): Macro-averaged precision.
            - recall (float): Macro-averaged recall.
            - f1 (float): Macro-averaged F1 score.
            - per_class_precision (list): Precision for each class.
            - per_class_recall (list): Recall for each class.
            - per_class_f1 (list): F1 score for each class.
            - cm (ndarray): Confusion matrix.
            - predictions (ndarray): Model predictions.
            - labels (ndarray): True labels.
    """
    # Set the model and classifier to evaluation mode
    model.eval()
    classifier.eval()

    # Initialize metrics
    total_loss = 0
    all_predictions = []
    all_labels = []
    total_samples = 0

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in dataloader:
            # Move batch data to the specified device
            premise_input_ids = batch["premise_input_ids"].to(device)
            premise_attention_mask = batch["premise_attention_mask"].to(device)
            premise_token_type_ids = batch["premise_token_type_ids"].to(device)
            hypothesis_input_ids = batch["hypothesis_input_ids"].to(device)
            hypothesis_attention_mask = batch["hypothesis_attention_mask"].to(device)
            hypothesis_token_type_ids = batch["hypothesis_token_type_ids"].to(device)
            labels = batch["labels"].to(device)

            # Get embeddings from the Bi-Encoder
            premise_embedding = model(premise_input_ids, premise_attention_mask, premise_token_type_ids)
            hypothesis_embedding = model(hypothesis_input_ids, hypothesis_attention_mask, hypothesis_token_type_ids)

            # Pass embeddings through the classification head
            logits = classifier(premise_embedding, hypothesis_embedding)

            # Calculate the loss
            loss = criterion(logits, labels)
            total_loss += loss.item() * premise_input_ids.size(0)

            # Get predictions
            _, predicted = torch.max(logits, dim=1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total_samples += labels.size(0)

    # Calculate evaluation metrics
    avg_loss = total_loss / total_samples
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro', zero_division=0)
    per_class_precision, per_class_recall, per_class_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average=None, zero_division=0)
    cm = confusion_matrix(all_labels, all_predictions)

    # Return metrics and predictions
    return avg_loss, accuracy, precision, recall, f1, per_class_precision, per_class_recall, per_class_f1, cm, np.array(all_predictions), np.array(all_labels)

print("Enhanced function 'evaluate_bi_encoder' defined (with predictions).")

In [ ]:
# ⚙️ Step 28: Implement the Full Bi-Encoder Training Loop
def train_bi_encoder(
    bi_encoder, bi_encoder_classifier, train_dataloader, val_dataloader, num_epochs, learning_rate, device, output_log_file
):
    """
    Trains the Bi-Encoder model with a classification head.

    Args:
        bi_encoder (nn.Module): The Bi-Encoder model.
        bi_encoder_classifier (nn.Module): The classification head for the Bi-Encoder.
        train_dataloader (DataLoader): DataLoader for the training dataset.
        val_dataloader (DataLoader): DataLoader for the validation dataset.
        num_epochs (int): Number of training epochs.
        learning_rate (float): Learning rate for the optimizer.
        device (torch.device): The device (CPU or GPU) to run the training on.
        output_log_file (str): Path to the log file for recording training progress.

    Returns:
        None
    """
    # Move models to the device
    bi_encoder.to(device)
    bi_encoder_classifier.to(device)

    # Define optimizer and scheduler
    optimizer = AdamW(list(bi_encoder.parameters()) + list(bi_encoder_classifier.parameters()), lr=learning_rate)
    total_steps = len(train_dataloader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    # Define loss function
    criterion = nn.CrossEntropyLoss()

    print(f"Training on: {device}")

    # Open log file for writing
    with open(output_log_file, "w") as f:
        f.write("Epoch,Step,Loss,Time\n")
        f.write("Epoch,Validation Loss,Validation Accuracy,Validation Precision (Macro),Validation Recall (Macro),Validation F1 (Macro)\n")

    # Training loop
    for epoch in range(num_epochs):
        start_time = time.time()
        epoch_total_loss = 0
        num_steps = 0
        progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch+1}/{num_epochs}")
        for i, batch in progress_bar:
            optimizer.zero_grad()

            # Move batch data to the specified device
            premise_input_ids = batch["premise_input_ids"].to(device)
            premise_attention_mask = batch["premise_attention_mask"].to(device)
            premise_token_type_ids = batch["premise_token_type_ids"].to(device)
            hypothesis_input_ids = batch["hypothesis_input_ids"].to(device)
            hypothesis_attention_mask = batch["hypothesis_attention_mask"].to(device)
            hypothesis_token_type_ids = batch["hypothesis_token_type_ids"].to(device)
            labels = batch["labels"].to(device)

            # Forward pass
            premise_embedding = bi_encoder(premise_input_ids, premise_attention_mask, premise_token_type_ids)
            hypothesis_embedding = bi_encoder(hypothesis_input_ids, hypothesis_attention_mask, hypothesis_token_type_ids)
            logits = bi_encoder_classifier(premise_embedding, hypothesis_embedding)

            # Compute loss and backpropagate
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Update metrics
            epoch_total_loss += loss.item()
            num_steps += 1
            progress_bar.set_postfix({"loss": f"{loss:.4f}"})

        # Evaluate on the validation set at the end of each epoch
        val_loss, val_accuracy, val_precision, val_recall, val_f1, _, _, _, _ = evaluate_bi_encoder(
            bi_encoder, bi_encoder_classifier, val_dataloader, criterion, device
        )
        avg_epoch_loss = epoch_total_loss / num_steps
        epoch_time = time.time() - start_time

        # Print epoch summary
        print(f"\n--- Epoch {epoch+1}/{num_epochs} Summary ---")
        print(f"Average Training Loss: {avg_epoch_loss:.4f}")
        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Accuracy: {val_accuracy:.4f}")
        print(f"Validation Precision (Macro): {val_precision:.4f}")
        print(f"Validation Recall (Macro): {val_recall:.4f}")
        print(f"Validation F1 (Macro): {val_f1:.4f}")
        print(f"Epoch Time: {epoch_time:.2f}s")

        # Log epoch summary to file
        with open(output_log_file, "a") as f:
            f.write(f"{epoch+1},{val_loss:.4f},{val_accuracy:.4f},{val_precision:.4f},{val_recall:.4f},{val_f1:.4f}\n")

    print("\nBi-Encoder Training Completed! Training details logged to:", output_log_file)


# Example usage
train_bi_encoder(
    bi_encoder=bi_encoder,
    bi_encoder_classifier=bi_encoder_classifier,
    train_dataloader=bi_encoder_train_dataloader,
    val_dataloader=bi_encoder_val_dataloader,
    num_epochs=3,
    learning_rate=5e-5,
    device=device,
    output_log_file="bi_encoder_training_log.txt",
)

In [ ]:
# ⚙️ Step 29: Evaluate Bi-Encoder on MNLI Validation Set
def evaluate_and_print_results(model, classifier, dataloader, criterion, device, dataset_name="Validation"):
    """
    Evaluates the Bi-Encoder model and prints the results.

    Args:
        model (nn.Module): The Bi-Encoder model.
        classifier (nn.Module): The classification head for the Bi-Encoder.
        dataloader (DataLoader): DataLoader for the evaluation dataset.
        criterion (nn.Module): Loss function for calculating the evaluation loss.
        device (torch.device): The device (CPU or GPU) to run the evaluation on.
        dataset_name (str): Name of the dataset being evaluated.

    Returns:
        dict: A dictionary containing evaluation metrics.
    """
    # Evaluate the model
    val_loss, val_accuracy, val_precision, val_recall, val_f1, val_pc_precision, val_pc_recall, val_pc_f1, val_cm, _, _ = evaluate_bi_encoder(
        model, classifier, dataloader, criterion, device
    )

    # Print evaluation results
    print(f"\n--- Bi-Encoder Evaluation Results ({dataset_name}) ---")
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Macro-averaged Precision: {val_precision:.4f}")
    print(f"Macro-averaged Recall: {val_recall:.4f}")
    print(f"Macro-averaged F1-Score: {val_f1:.4f}")
    print("\nPer-Class Metrics:")
    for i, label in enumerate(["Entailment", "Neutral", "Contradiction"]):
        print(f"  {label}: Precision={val_pc_precision[i]:.4f}, Recall={val_pc_recall[i]:.4f}, F1-Score={val_pc_f1[i]:.4f}")
    print("\nConfusion Matrix:")
    print(val_cm)

    # Return metrics as a dictionary
    return {
        "loss": val_loss,
        "accuracy": val_accuracy,
        "precision": val_precision,
        "recall": val_recall,
        "f1": val_f1,
        "per_class_precision": val_pc_precision,
        "per_class_recall": val_pc_recall,
        "per_class_f1": val_pc_f1,
        "confusion_matrix": val_cm,
    }


# Evaluate on MNLI Validation Set
mnli_validation_results = evaluate_and_print_results(
    bi_encoder, bi_encoder_classifier, bi_encoder_val_dataloader, criterion, device, dataset_name="MNLI Validation"
)

In [ ]:
# ⚙️ Step 30: Access Spanish XNLI Test Set
def access_test_set(dataset, language="Spanish"):
    """
    Accesses the test set for a given language.

    Args:
        dataset (Dataset): The dataset containing the test set.
        language (str): The language of the test set.

    Returns:
        Dataset: The test set for the specified language.
    """
    try:
        test_set = dataset["test"]
        print(f"{language} XNLI Test Set accessed with {len(test_set)} samples.")
        print(test_set[0])  # Print the first sample for inspection
        return test_set
    except KeyError as e:
        print(f"Error accessing {language} XNLI Test Set: {e}")
        raise


# Access Spanish XNLI Test Set
xnli_es_test = access_test_set(xnli_es_dataset, language="Spanish")

In [ ]:
# ⚙️ Step 31: Access German XNLI Test Set
def access_test_set(dataset, language="German"):
    """
    Accesses the test set for a given language.

    Args:
        dataset (Dataset): The dataset containing the test set.
        language (str): The language of the test set.

    Returns:
        Dataset: The test set for the specified language.
    """
    try:
        test_set = dataset["test"]
        print(f"{language} XNLI Test Set accessed with {len(test_set)} samples.")
        print(test_set[0])  # Print the first sample for inspection
        return test_set
    except KeyError as e:
        print(f"Error accessing {language} XNLI Test Set: {e}")
        raise


# Access German XNLI Test Set
xnli_de_test = access_test_set(xnli_de_dataset, language="German")

In [ ]:
## check first element of Spanish XNLI test set
# print(xnli_es_test[0])

In [ ]:
# ⚙️ Step 32: Preprocess XNLI Test Set for Bi-Encoder
def preprocess_xnli_bi_encoder(examples, premise_key="premise", hypothesis_key="hypothesis"):
    """
    Preprocesses XNLI test set for the Bi-Encoder.

    Args:
        examples (dict): A batch of examples containing premise and hypothesis.
        premise_key (str): Key for the premise field in the dataset.
        hypothesis_key (str): Key for the hypothesis field in the dataset.

    Returns:
        dict: Preprocessed and tokenized inputs for the Bi-Encoder.
    """
    premise = examples[premise_key]
    hypothesis = examples[hypothesis_key]
    encoded_premise = tokenizer(premise, truncation=True, padding="max_length", max_length=100)
    encoded_hypothesis = tokenizer(hypothesis, truncation=True, padding="max_length", max_length=100)
    return {
        "premise_input_ids": encoded_premise["input_ids"],
        "premise_attention_mask": encoded_premise["attention_mask"],
        "premise_token_type_ids": encoded_premise.get("token_type_ids", [[0] * len(ids) for ids in encoded_premise["input_ids"]]),
        "hypothesis_input_ids": encoded_hypothesis["input_ids"],
        "hypothesis_attention_mask": encoded_hypothesis["attention_mask"],
        "hypothesis_token_type_ids": encoded_hypothesis.get("token_type_ids", [[0] * len(ids) for ids in encoded_hypothesis["input_ids"]]),
        "labels": examples["label"]
    }


# Preprocess Spanish XNLI Test Set
xnli_es_encoded_test = xnli_es_test.map(preprocess_xnli_bi_encoder, batched=True)
xnli_es_encoded_test.set_format("torch", columns=["premise_input_ids", "premise_attention_mask", "premise_token_type_ids", "hypothesis_input_ids", "hypothesis_attention_mask", "hypothesis_token_type_ids", "labels"])

print("Spanish XNLI Test Set preprocessed and formatted for Bi-Encoder.")
print(xnli_es_encoded_test[0])

In [ ]:
# ⚙️ Step 33: Preprocess German XNLI Test Set for Bi-Encoder
# Reuse the `preprocess_xnli_bi_encoder` function defined earlier
xnli_de_encoded_test = xnli_de_test.map(preprocess_xnli_bi_encoder, batched=True)

# Set the format for PyTorch tensors and specify the columns to return
xnli_de_encoded_test.set_format(
    "torch",
    columns=[
        "premise_input_ids",
        "premise_attention_mask",
        "premise_token_type_ids",
        "hypothesis_input_ids",
        "hypothesis_attention_mask",
        "hypothesis_token_type_ids",
        "labels",
    ],
)

# Print confirmation and inspect the first preprocessed sample
print("German XNLI Test Set preprocessed and formatted for Bi-Encoder.")
print(xnli_de_encoded_test[0])

In [ ]:
# ⚙️ Step 34: Create DataLoaders for Spanish and German XNLI Test Sets
# Define evaluation batch size
eval_batch_size = 32  # Adjust based on available memory and hardware

# Create DataLoaders for Spanish and German XNLI Test Sets
bi_encoder_es_test_dataloader = DataLoader(
    xnli_es_encoded_test, batch_size=eval_batch_size, shuffle=False
)
bi_encoder_de_test_dataloader = DataLoader(
    xnli_de_encoded_test, batch_size=eval_batch_size, shuffle=False
)

# Print confirmation of DataLoader creation
print(f"DataLoader for Spanish XNLI Test Set created with {len(bi_encoder_es_test_dataloader)} batches.")
print(f"DataLoader for German XNLI Test Set created with {len(bi_encoder_de_test_dataloader)} batches.")

In [ ]:
# ⚙️ Step 35: Evaluate Bi-Encoder on Spanish and German XNLI Test Sets
def print_evaluation_results(language, loss, accuracy, precision, recall, f1, per_class_precision, per_class_recall, per_class_f1, cm):
    """
    Prints evaluation results for a given dataset.

    Args:
        language (str): The language of the dataset (e.g., "Spanish", "German").
        loss (float): Test loss.
        accuracy (float): Test accuracy.
        precision (float): Macro-averaged precision.
        recall (float): Macro-averaged recall.
        f1 (float): Macro-averaged F1 score.
        per_class_precision (list): Precision for each class.
        per_class_recall (list): Recall for each class.
        per_class_f1 (list): F1 score for each class.
        cm (ndarray): Confusion matrix.
    """
    print(f"\n--- Bi-Encoder Evaluation Results ({language} XNLI Test Set) ---")
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Macro-averaged Precision: {precision:.4f}")
    print(f"Macro-averaged Recall: {recall:.4f}")
    print(f"Macro-averaged F1-Score: {f1:.4f}")
    print("\nPer-Class Metrics:")
    for i, label in enumerate(["Entailment", "Neutral", "Contradiction"]):
        print(f"  {label}: Precision={per_class_precision[i]:.4f}, Recall={per_class_recall[i]:.4f}, F1-Score={per_class_f1[i]:.4f}")
    print("\nConfusion Matrix:")
    print(cm)

# Evaluate Bi-Encoder on Spanish XNLI Test Set
es_test_loss, es_test_accuracy, es_precision, es_recall, es_f1, es_pc_precision, es_pc_recall, es_pc_f1, es_cm, es_bi_encoder_predictions, es_bi_encoder_labels = evaluate_bi_encoder(
    bi_encoder, bi_encoder_classifier, bi_encoder_es_test_dataloader, criterion, device
)
print_evaluation_results("Spanish", es_test_loss, es_test_accuracy, es_precision, es_recall, es_f1, es_pc_precision, es_pc_recall, es_pc_f1, es_cm)

# Evaluate Bi-Encoder on German XNLI Test Set
de_test_loss, de_test_accuracy, de_precision, de_recall, de_f1, de_pc_precision, de_pc_recall, de_pc_f1, de_cm, de_bi_encoder_predictions, de_bi_encoder_labels = evaluate_bi_encoder(
    bi_encoder, bi_encoder_classifier, bi_encoder_de_test_dataloader, criterion, device
)
print_evaluation_results("German", de_test_loss, de_test_accuracy, de_precision, de_recall, de_f1, de_pc_precision, de_pc_recall, de_pc_f1, de_cm)

In [ ]:
# ⚙️ Step 36: Re-evaluate Bi-Encoder on MNLI Validation with Enhanced Metrics
print("\n--- Re-evaluating Bi-Encoder on MNLI Validation (Enhanced Metrics) ---")

# Perform evaluation on the MNLI validation set
val_loss, val_accuracy, val_precision, val_recall, val_f1, val_pc_precision, val_pc_recall, val_pc_f1, val_cm, val_bi_encoder_predictions_again, val_bi_encoder_labels_again = evaluate_bi_encoder(
    bi_encoder, bi_encoder_classifier, bi_encoder_val_dataloader, criterion, device
)

# Print evaluation results
print_evaluation_results(
    "MNLI Validation",
    val_loss,
    val_accuracy,
    val_precision,
    val_recall,
    val_f1,
    val_pc_precision,
    val_pc_recall,
    val_pc_f1,
    val_cm,
)

# Print shapes of predictions and labels for verification
print(f"Bi-Encoder Validation Predictions Shape (Re-evaluation): {val_bi_encoder_predictions_again.shape}")
print(f"Bi-Encoder Validation True Labels Shape (Re-evaluation): {val_bi_encoder_labels_again.shape}")

In [ ]:
# ⚙️ Step 37: Retrain Cross-Encoder on Full MNLI Dataset
print("\n--- Retraining Cross-Encoder on Full MNLI Dataset ---")

# Re-initialize the cross-encoder model
cross_encoder_retrained = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", num_labels=3
).to(device)

# Define training arguments for retraining
training_args_retrained = TrainingArguments(
    output_dir="./cross_encoder_retrained",  # Directory to save the retrained model
    num_train_epochs=3,                      # Number of training epochs (matching bi-encoder)
    per_device_train_batch_size=64,          # Batch size for training
    gradient_accumulation_steps=1,           # Gradient accumulation steps
    learning_rate=2e-5,                      # Learning rate
    weight_decay=0.01,                       # Weight decay for regularization
    logging_dir="./logs_cross_encoder_retrained",  # Directory for logging
    logging_steps=500,                       # Log every 500 steps
    evaluation_strategy="epoch",            # Evaluate at the end of each epoch
    save_strategy="epoch",                  # Save model at the end of each epoch
    load_best_model_at_end=False,           # Do not load the best model automatically
    metric_for_best_model="eval_loss",      # Metric to determine the best model
    report_to="none"                        # Disable reporting to external services
)

# Initialize the Trainer for retraining
trainer_retrained = Trainer(
    model=cross_encoder_retrained,
    args=training_args_retrained,
    train_dataset=mnli_encoded_train,       # Full MNLI training dataset
    eval_dataset=mnli_encoded_validation,  # MNLI validation dataset
    tokenizer=tokenizer,                   # Tokenizer for preprocessing
    compute_metrics=compute_metrics,       # Evaluation metrics
)

# Train the cross-encoder model
trainer_retrained.train()

# Save the retrained model
trainer_retrained.save_model("./cross_encoder_retrained_final")
print("--- Cross-Encoder Retraining Complete and Model Saved ---")

In [ ]:
# ⚙️ Step 38: Evaluate Retrained Cross-Encoder on MNLI and XNLI Test Sets
def evaluate_cross_encoder(model, dataloader, criterion, device):
    """
    Evaluates the Cross-Encoder model on a given dataset.

    Args:
        model (nn.Module): The Cross-Encoder model.
        dataloader (DataLoader): DataLoader for the evaluation dataset.
        criterion (nn.Module): Loss function for calculating the evaluation loss.
        device (torch.device): The device (CPU or GPU) to run the evaluation on.

    Returns:
        tuple: A tuple containing evaluation metrics and predictions.
    """
    model.eval()
    total_loss = 0
    all_predictions = []
    all_labels = []
    total_samples = 0

    with torch.no_grad():
        for batch in dataloader:
            # Move batch data to the specified device
            batch = {k: v.to(device) for k, v in batch.items()}

            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            token_type_ids = batch.get("token_type_ids", None)
            labels = batch["labels"]

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            logits = outputs.logits

            # Calculate loss
            loss = criterion(logits, labels)
            total_loss += loss.item() * input_ids.size(0)

            # Get predictions
            _, predicted = torch.max(logits, dim=1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total_samples += labels.size(0)

    # Calculate evaluation metrics
    avg_loss = total_loss / total_samples
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro', zero_division=0)
    per_class_precision, per_class_recall, per_class_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average=None, zero_division=0)
    cm = confusion_matrix(all_labels, all_predictions)

    return avg_loss, accuracy, precision, recall, f1, per_class_precision, per_class_recall, per_class_f1, cm, np.array(all_predictions), np.array(all_labels)

# Evaluate on MNLI Validation Set
print("\n--- Evaluating Retrained Cross-Encoder on MNLI Validation ---")
ce_val_loss, ce_val_accuracy, ce_val_precision, ce_val_recall, ce_val_f1, ce_val_pc_precision, ce_val_pc_recall, ce_val_pc_f1, ce_val_cm, ce_val_predictions, ce_val_labels = evaluate_cross_encoder(
    cross_encoder_retrained, cross_encoder_val_dataloader, cross_encoder_criterion, device
)
print_evaluation_results("MNLI Validation (Cross-Encoder)", ce_val_loss, ce_val_accuracy, ce_val_precision, ce_val_recall, ce_val_f1, ce_val_pc_precision, ce_val_pc_recall, ce_val_pc_f1, ce_val_cm)

# Evaluate on Spanish XNLI Test Set
print("\n--- Evaluating Retrained Cross-Encoder on Spanish XNLI Test Set ---")
ce_es_test_loss, ce_es_test_accuracy, ce_es_test_precision, ce_es_test_recall, ce_es_test_f1, ce_es_test_pc_precision, ce_es_test_pc_recall, ce_es_test_pc_f1, ce_es_test_cm, ce_es_test_predictions, ce_es_test_labels = evaluate_cross_encoder(
    cross_encoder_retrained, cross_encoder_es_test_dataloader, cross_encoder_criterion, device
)
print_evaluation_results("Spanish XNLI Test (Cross-Encoder)", ce_es_test_loss, ce_es_test_accuracy, ce_es_test_precision, ce_es_test_recall, ce_es_test_f1, ce_es_test_pc_precision, ce_es_test_pc_recall, ce_es_test_pc_f1, ce_es_test_cm)

# Evaluate on German XNLI Test Set
print("\n--- Evaluating Retrained Cross-Encoder on German XNLI Test Set ---")
ce_de_test_loss, ce_de_test_accuracy, ce_de_test_precision, ce_de_test_recall, ce_de_test_f1, ce_de_test_pc_precision, ce_de_test_pc_recall, ce_de_test_pc_f1, ce_de_test_cm, ce_de_test_predictions, ce_de_test_labels = evaluate_cross_encoder(
    cross_encoder_retrained, cross_encoder_de_test_dataloader, cross_encoder_criterion, device
)
print_evaluation_results("German XNLI Test (Cross-Encoder)", ce_de_test_loss, ce_de_test_accuracy, ce_de_test_precision, ce_de_test_recall, ce_de_test_f1, ce_de_test_pc_precision, ce_de_test_pc_recall, ce_de_test_pc_f1, ce_de_test_cm)

In [ ]:
# ⚙️ Step 39: Collect and Organize Results
print("\n--- Collecting and Organizing Results ---")

# Organize Bi-Encoder results
bi_encoder_results = {
    "MNLI Validation": {
        "loss": val_loss,
        "accuracy": val_accuracy,
        "precision": val_precision,
        "recall": val_recall,
        "f1": val_f1,
        "per_class_precision": val_pc_precision,
        "per_class_recall": val_pc_recall,
        "per_class_f1": val_pc_f1,
        "confusion_matrix": val_cm,
    },
    "Spanish XNLI Test": {
        "loss": es_test_loss,
        "accuracy": es_test_accuracy,
        "precision": es_precision,
        "recall": es_recall,
        "f1": es_f1,
        "per_class_precision": es_pc_precision,
        "per_class_recall": es_pc_recall,
        "per_class_f1": es_pc_f1,
        "confusion_matrix": es_cm,
    },
    "German XNLI Test": {
        "loss": de_test_loss,
        "accuracy": de_test_accuracy,
        "precision": de_precision,
        "recall": de_recall,
        "f1": de_f1,
        "per_class_precision": de_pc_precision,
        "per_class_recall": de_pc_recall,
        "per_class_f1": de_pc_f1,
        "confusion_matrix": de_cm,
    },
}

# Organize Cross-Encoder results
cross_encoder_results = {
    "MNLI Validation": {
        "loss": ce_val_loss,
        "accuracy": ce_val_accuracy,
        "precision": ce_val_precision,
        "recall": ce_val_recall,
        "f1": ce_val_f1,
        "per_class_precision": ce_val_pc_precision,
        "per_class_recall": ce_val_pc_recall,
        "per_class_f1": ce_val_pc_f1,
        "confusion_matrix": ce_val_cm,
    },
    "Spanish XNLI Test": {
        "loss": ce_es_test_loss,
        "accuracy": ce_es_test_accuracy,
        "precision": ce_es_test_precision,
        "recall": ce_es_test_recall,
        "f1": ce_es_test_f1,
        "per_class_precision": ce_es_test_pc_precision,
        "per_class_recall": ce_es_test_pc_recall,
        "per_class_f1": ce_es_test_pc_f1,
        "confusion_matrix": ce_es_test_cm,
    },
    "German XNLI Test": {
        "loss": ce_de_test_loss,
        "accuracy": ce_de_test_accuracy,
        "precision": ce_de_test_precision,
        "recall": ce_de_test_recall,
        "f1": ce_de_test_f1,
        "per_class_precision": ce_de_test_pc_precision,
        "per_class_recall": ce_de_test_pc_recall,
        "per_class_f1": ce_de_test_pc_f1,
        "confusion_matrix": ce_de_test_cm,
    },
}

# Convert results to Pandas DataFrames for easier analysis
bi_encoder_df = pd.DataFrame.from_dict(
    {(dataset, metric): bi_encoder_results[dataset][metric]
     for dataset in bi_encoder_results.keys()
     for metric in bi_encoder_results[dataset].keys()},
    orient="index",
)

cross_encoder_df = pd.DataFrame.from_dict(
    {(dataset, metric): cross_encoder_results[dataset][metric]
     for dataset in cross_encoder_results.keys()
     for metric in cross_encoder_results[dataset].keys()},
    orient="index",
)

# Display the DataFrames (will render as HTML tables in Jupyter)
print("\n--- Bi-Encoder Results DataFrame ---")
display(bi_encoder_df)

print("\n--- Cross-Encoder Results DataFrame ---")
display(cross_encoder_df)

In [ ]:
# ⚙️ Step 40: Analyze and Compare Performance
# Assuming bi_encoder_results and cross_encoder_results dictionaries are populated

def compare_models(bi_results, ce_results, dataset_name):
    """
    Compares the performance of Bi-Encoder and Cross-Encoder models on a specific dataset.

    Args:
        bi_results (dict): Results dictionary for the Bi-Encoder model.
        ce_results (dict): Results dictionary for the Cross-Encoder model.
        dataset_name (str): Name of the dataset being compared (e.g., "MNLI Validation").

    Returns:
        None
    """
    print(f"\n--- Performance Comparison: {dataset_name} ---")
    metrics_to_compare = ["accuracy", "precision", "recall", "f1"]
    print(f"{'Metric':<15} {'Bi-Encoder':<15} {'Cross-Encoder':<15} {'Difference':<15}")
    for metric in metrics_to_compare:
        bi_val = bi_results[dataset_name][metric]
        ce_val = ce_results[dataset_name][metric]
        difference = ce_val - bi_val
        print(f"{metric:<15} {bi_val:<15.4f} {ce_val:<15.4f} {difference:<15.4f}")

    print("\n--- Per-Class F1-Score Comparison ---")
    classes = ["Entailment", "Neutral", "Contradiction"]
    print(f"{'Class':<15} {'Bi-Encoder':<15} {'Cross-Encoder':<15} {'Difference':<15}")
    for i, cls in enumerate(classes):
        bi_f1 = bi_results[dataset_name]["per_class_f1"][i]
        ce_f1 = ce_results[dataset_name]["per_class_f1"][i]
        difference = ce_f1 - bi_f1
        print(f"{cls:<15} {bi_f1:<15.4f} {ce_f1:<15.4f} {difference:<15.4f}")

    print("\n--- Confusion Matrix (Cross-Encoder vs. Bi-Encoder) ---")
    print("\nBi-Encoder Confusion Matrix:")
    print(bi_results[dataset_name]["confusion_matrix"])
    print("\nCross-Encoder Confusion Matrix:")
    print(ce_results[dataset_name]["confusion_matrix"])

# Compare on MNLI Validation
compare_models(bi_encoder_results, cross_encoder_results, "MNLI Validation")

# Compare on Spanish XNLI Test
compare_models(bi_encoder_results, cross_encoder_results, "Spanish XNLI Test")

# Compare on German XNLI Test
compare_models(bi_encoder_results, cross_encoder_results, "German XNLI Test")

In [ ]:
# ⚙️ Step 41: Statistical Significance Testing with More Iterations
def perform_bootstrap_test(predictions_model1, predictions_model2, true_labels, metric=accuracy_score, n_iterations=10000, alpha=0.05):
    """
    Performs a paired bootstrap hypothesis test to compare the performance of two models.

    Args:
        predictions_model1 (np.array): Predictions of the first model.
        predictions_model2 (np.array): Predictions of the second model.
        true_labels (np.array): True labels.
        metric (callable): Scoring function (e.g., accuracy_score).
        n_iterations (int): Number of bootstrap iterations.
        alpha (float): Significance level.

    Returns:
        dict: A dictionary containing the observed difference, bootstrap differences,
              p-value (two-tailed), and the confidence interval of the difference.
    """
    n_samples = len(true_labels)
    observed_difference = metric(true_labels, predictions_model2) - metric(true_labels, predictions_model1)
    bootstrap_differences = []

    for _ in range(n_iterations):
        # Resample with replacement
        indices = resample(np.arange(n_samples), n_samples=n_samples, replace=True)
        bootstrap_preds1 = predictions_model1[indices]
        bootstrap_preds2 = predictions_model2[indices]
        bootstrap_labels = true_labels[indices]

        diff = metric(bootstrap_labels, bootstrap_preds2) - metric(bootstrap_labels, bootstrap_preds1)
        bootstrap_differences.append(diff)

    bootstrap_differences = np.array(bootstrap_differences)
    p_value = np.mean(np.abs(bootstrap_differences) >= np.abs(observed_difference))
    confidence_interval = np.percentile(bootstrap_differences, [(alpha / 2) * 100, (1 - alpha / 2) * 100])

    return {
        "observed_difference": observed_difference,
        "bootstrap_differences": bootstrap_differences,
        "p_value": p_value,
        "confidence_interval": confidence_interval,
    }

# Perform bootstrap tests for accuracy on each dataset with increased iterations
print("\n--- Bootstrap Test (10000 iterations): Accuracy on MNLI Validation ---")
mnli_accuracy_results = perform_bootstrap_test(val_bi_encoder_predictions, ce_val_predictions, val_bi_encoder_labels)
print(f"Observed Accuracy Difference (Cross - Bi): {mnli_accuracy_results['observed_difference']:.4f}")
print(f"P-value: {mnli_accuracy_results['p_value']:.4f}")
print(f"95% Confidence Interval: [{mnli_accuracy_results['confidence_interval'][0]:.4f}, {mnli_accuracy_results['confidence_interval'][1]:.4f}]")

print("\n--- Bootstrap Test (10000 iterations): Accuracy on Spanish XNLI Test ---")
xnli_es_accuracy_results = perform_bootstrap_test(es_bi_encoder_predictions, ce_es_test_predictions, es_bi_encoder_labels)
print(f"Observed Accuracy Difference (Cross - Bi): {xnli_es_accuracy_results['observed_difference']:.4f}")
print(f"P-value: {xnli_es_accuracy_results['p_value']:.4f}")
print(f"95% Confidence Interval: [{xnli_es_accuracy_results['confidence_interval'][0]:.4f}, {xnli_es_accuracy_results['confidence_interval'][1]:.4f}]")

print("\n--- Bootstrap Test (10000 iterations): Accuracy on German XNLI Test ---")
xnli_de_accuracy_results = perform_bootstrap_test(de_bi_encoder_predictions, ce_de_test_predictions, de_bi_encoder_labels)
print(f"Observed Accuracy Difference (Cross - Bi): {xnli_de_accuracy_results['observed_difference']:.4f}")
print(f"P-value: {xnli_de_accuracy_results['p_value']:.4f}")
print(f"95% Confidence Interval: [{xnli_de_accuracy_results['confidence_interval'][0]:.4f}, {xnli_de_accuracy_results['confidence_interval'][1]:.4f}]")

In [ ]:
# ⚙️ Step 42: Further Data Analysis and Visualization

# Organize the results into a DataFrame for easier plotting
results_df = pd.DataFrame({
    'Model': ['Bi-Encoder', 'Cross-Encoder', 'Bi-Encoder', 'Cross-Encoder', 'Bi-Encoder', 'Cross-Encoder'],
    'Dataset': ['MNLI Validation', 'MNLI Validation', 'Spanish XNLI Test', 'Spanish XNLI Test', 'German XNLI Test', 'German XNLI Test'],
    'Accuracy': [val_accuracy, ce_val_accuracy, es_test_accuracy, ce_es_test_accuracy, de_test_accuracy, ce_de_test_accuracy],
    'F1-Score (Macro)': [val_f1, ce_val_f1, es_f1, ce_es_test_f1, de_f1, ce_de_test_f1]
})

# Calculate percentage difference in Accuracy
def calculate_percentage_difference(bi_val, cross_val):
    return ((cross_val - bi_val) / bi_val) * 100

mnli_acc_diff_percent = calculate_percentage_difference(val_accuracy, ce_val_accuracy)
es_acc_diff_percent = calculate_percentage_difference(es_test_accuracy, ce_es_test_accuracy)
de_acc_diff_percent = calculate_percentage_difference(de_test_accuracy, ce_de_test_accuracy)

print("\n--- Percentage Difference in Accuracy (Cross-Encoder vs. Bi-Encoder) ---")
print(f"MNLI Validation: +{mnli_acc_diff_percent:.2f}%")
print(f"Spanish XNLI Test: +{es_acc_diff_percent:.2f}%")
print(f"German XNLI Test: +{de_acc_diff_percent:.2f}%")

# --- Visualizations ---

# 1. Accuracy Comparison Bar Chart
plt.figure(figsize=(10, 6))
sns.barplot(x='Dataset', y='Accuracy', hue='Model', data=results_df)
plt.title('Accuracy Comparison: Bi-Encoder vs. Cross-Encoder')
plt.ylabel('Accuracy')
plt.xlabel('Dataset')
plt.ylim(0.5, 0.9)  # Adjust y-axis limits for better visualization
plt.show()

# 2. F1-Score (Macro) Comparison Bar Chart
plt.figure(figsize=(10, 6))
sns.barplot(x='Dataset', y='F1-Score (Macro)', hue='Model', data=results_df)
plt.title('Macro F1-Score Comparison: Bi-Encoder vs. Cross-Encoder')
plt.ylabel('Macro F1-Score')
plt.xlabel('Dataset')
plt.ylim(0.5, 0.9)  # Adjust y-axis limits for better visualization
plt.show()

# 3. Confusion Matrix Heatmaps (Optional - can be insightful for error patterns)
def plot_confusion_matrix(cm, labels, title):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

plot_confusion_matrix(val_cm, ["Entailment", "Neutral", "Contradiction"], 'Bi-Encoder Confusion Matrix (MNLI Val)')
plot_confusion_matrix(ce_val_cm, ["Entailment", "Neutral", "Contradiction"], 'Cross-Encoder Confusion Matrix (MNLI Val)')
plot_confusion_matrix(es_cm, ["Entailment", "Neutral", "Contradiction"], 'Bi-Encoder Confusion Matrix (Spanish XNLI)')
plot_confusion_matrix(ce_es_test_cm, ["Entailment", "Neutral", "Contradiction"], 'Cross-Encoder Confusion Matrix (Spanish XNLI)')
plot_confusion_matrix(de_cm, ["Entailment", "Neutral", "Contradiction"], 'Bi-Encoder Confusion Matrix (German XNLI)')
plot_confusion_matrix(ce_de_test_cm, ["Entailment", "Neutral", "Contradiction"], 'Cross-Encoder Confusion Matrix (German XNLI)')

In [ ]:
# ⚙️ Step 43: Output Performance Metrics in Text Tables (Optional)
def output_performance_metrics():
    """
    Outputs performance metrics for Bi-Encoder and Cross-Encoder models in text tables.
    This step is optional and can be called if detailed text-based comparisons are needed.
    """
    print("\n--- Performance Comparison: MNLI Validation ---")
    print(f"{'Metric':<10}  {'Bi-Encoder':<15}  {'Cross-Encoder':<15}  {'Difference':<10}")
    print(f"{'Accuracy':<10}  {val_accuracy:<15.4f}  {ce_val_accuracy:<15.4f}  {ce_val_accuracy - val_accuracy:<10.4f}")
    print(f"{'Precision':<10}  {val_precision:<15.4f}  {ce_val_precision:<15.4f}  {ce_val_precision - val_precision:<10.4f}")
    print(f"{'Recall':<10}     {val_recall:<15.4f}  {ce_val_recall:<15.4f}  {ce_val_recall - val_recall:<10.4f}")
    print(f"{'F1':<10}         {val_f1:<15.4f}  {ce_val_f1:<15.4f}  {ce_val_f1 - val_f1:<10.4f}")

    print("\n--- Per-Class F1-Score Comparison: MNLI Validation ---")
    print(f"{'Class':<15}  {'Bi-Encoder':<15}  {'Cross-Encoder':<15}  {'Difference':<10}")
    for i, label in enumerate(["Entailment", "Neutral", "Contradiction"]):
        print(f"{label:<15}  {val_pc_f1[i]:<15.4f}  {ce_val_pc_f1[i]:<15.4f}  {ce_val_pc_f1[i] - val_pc_f1[i]:<10.4f}")

    print("\n--- Performance Comparison: Spanish XNLI Test ---")
    print(f"{'Metric':<10}  {'Bi-Encoder':<15}  {'Cross-Encoder':<15}  {'Difference':<10}")
    print(f"{'Accuracy':<10}  {es_test_accuracy:<15.4f}  {ce_es_test_accuracy:<15.4f}  {ce_es_test_accuracy - es_test_accuracy:<10.4f}")
    print(f"{'Precision':<10}  {es_precision:<15.4f}  {ce_es_test_precision:<15.4f}  {ce_es_test_precision - es_precision:<10.4f}")
    print(f"{'Recall':<10}     {es_recall:<15.4f}  {ce_es_test_recall:<15.4f}  {ce_es_test_recall - es_recall:<10.4f}")
    print(f"{'F1':<10}         {es_f1:<15.4f}  {ce_es_test_f1:<15.4f}  {ce_es_test_f1 - es_f1:<10.4f}")

    print("\n--- Per-Class F1-Score Comparison: Spanish XNLI Test ---")
    print(f"{'Class':<15}  {'Bi-Encoder':<15}  {'Cross-Encoder':<15}  {'Difference':<10}")
    for i, label in enumerate(["Entailment", "Neutral", "Contradiction"]):
        print(f"{label:<15}  {es_pc_f1[i]:<15.4f}  {ce_es_test_pc_f1[i]:<15.4f}  {ce_es_test_pc_f1[i] - es_pc_f1[i]:<10.4f}")

    print("\n--- Performance Comparison: German XNLI Test ---")
    print(f"{'Metric':<10}  {'Bi-Encoder':<15}  {'Cross-Encoder':<15}  {'Difference':<10}")
    print(f"{'Accuracy':<10}  {de_test_accuracy:<15.4f}  {ce_de_test_accuracy:<15.4f}  {ce_de_test_accuracy - de_test_accuracy:<10.4f}")
    print(f"{'Precision':<10}  {de_precision:<15.4f}  {ce_de_test_precision:<15.4f}  {ce_de_test_precision - de_precision:<10.4f}")
    print(f"{'Recall':<10}     {de_recall:<15.4f}  {ce_de_test_recall:<15.4f}  {ce_de_test_recall - de_recall:<10.4f}")
    print(f"{'F1':<10}         {de_f1:<15.4f}  {ce_de_test_f1:<15.4f}  {ce_de_test_f1 - de_f1:<10.4f}")

    print("\n--- Per-Class F1-Score Comparison: German XNLI Test ---")
    print(f"{'Class':<15}  {'Bi-Encoder':<15}  {'Cross-Encoder':<15}  {'Difference':<10}")
    for i, label in enumerate(["Entailment", "Neutral", "Contradiction"]):
        print(f"{label:<15}  {de_pc_f1[i]:<15.4f}  {ce_de_test_pc_f1[i]:<15.4f}  {ce_de_test_pc_f1[i] - de_pc_f1[i]:<10.4f}")

    print("\n--- Statistical Significance (Accuracy) ---")
    print(f"\nMNLI Validation:")
    print(f"  Observed Difference (Cross - Bi): {mnli_accuracy_results['observed_difference']:.4f}")
    print(f"  P-value: {mnli_accuracy_results['p_value']:.4f}")
    print(f"  95% Confidence Interval: [{mnli_accuracy_results['confidence_interval'][0]:.4f}, {mnli_accuracy_results['confidence_interval'][1]:.4f}]")

    print(f"\nSpanish XNLI Test:")
    print(f"  Observed Difference (Cross - Bi): {xnli_es_accuracy_results['observed_difference']:.4f}")
    print(f"  P-value: {xnli_es_accuracy_results['p_value']:.4f}")
    print(f"  95% Confidence Interval: [{xnli_es_accuracy_results['confidence_interval'][0]:.4f}, {xnli_es_accuracy_results['confidence_interval'][1]:.4f}]")

    print(f"\nGerman XNLI Test:")
    print(f"  Observed Difference (Cross - Bi): {xnli_de_accuracy_results['observed_difference']:.4f}")
    print(f"  P-value: {xnli_de_accuracy_results['p_value']:.4f}")
    print(f"  95% Confidence Interval: [{xnli_de_accuracy_results['confidence_interval'][0]:.4f}, {xnli_de_accuracy_results['confidence_interval'][1]:.4f}]")

    print("\n--- Percentage Difference in Accuracy (Cross-Encoder vs. Bi-Encoder) ---")
    print(f"MNLI Validation: +{mnli_acc_diff_percent:.2f}%")
    print(f"Spanish XNLI Test: +{es_acc_diff_percent:.2f}%")
    print(f"German XNLI Test: +{de_acc_diff_percent:.2f}%")

# Uncomment the following line to run this optional step
# output_performance_metrics()